# Colab-DFDNet-lightning

Official repo: [csxmli2016/DFDNet](https://github.com/csxmli2016/DFDNet)

Original repo: [max-vasyuk/DFDNet](https://github.com/max-vasyuk/DFDNet)

My fork: [styler00dollar/Colab-DFDNet](https://github.com/styler00dollar/Colab-DFDNet)

Porting everything to pytorch lightning. And storing code within Github. For more instructions, view my [other Colab](https://github.com/styler00dollar/Colab-DFDNet/blob/master/Colab-DFDNet-lightning-train.ipynb).

Preconfigured paths:
```
/content/DFDNet/DictionaryCenter512 (numpy dictionary files)
/content/DFDNet/ffhq (dataset path)
/content/DFDNet/landmarks (landmarks for that dataset)
/content/DFDNet/weights/vgg19.pth (vgg19 path)

/content/validation (validation path)
/content/landmarks (landmark path for validation)
```
By default it will assume, that 1024px is used for training and 1024px is also used for validation. ``self.size`` is 512px and shouldn't be changed. If you don't want to apply rescaling to validation, change 
```
part_locations = self.get_part_location(self.partpath, ImgName, 2)
``` 
inside the validation dataloader. 2 is the scaling factor for landmarks.

Edit ``vgg path`` and ``dictionary_path`` inside ``model.py``. Inside ``CustomTrainClass.py`` it is possible to configure logging path, loss functions and weights.

Warning: Don't use AMP with StyleLoss. Perceptual loss will have problems with multi-GPU.

Restart with ``runtime > restart runtime`` once if you see ``AttributeError: module 'PIL.TiffTags' has no attribute 'IFD'``.


In [ ]:
!nvidia-smi

In [ ]:
#@title GPU
!pip install pytorch-lightning -U
!git clone -b local --single-branch https://github.com/styler00dollar/Colab-DFDNet DFDNet
%cd /content/DFDNet
!pip install -r requirements.txt
!pip install pytorch-msssim
!pip install trains
!pip install PyJWT==1.7.1
!pip install tensorboardX

In [ ]:
#@title TPU  (restart runtime afterwards)
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
#!pip install pytorch-lightning
!pip install lightning-flash

import collections
from datetime import datetime, timedelta
import os
import requests
import threading

_VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
VERSION = "xrt==1.15.0"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
CONFIG = {
    'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
    'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
        (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
}[VERSION]
DIST_BUCKET = 'gs://tpu-pytorch/wheels'
TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# Update TPU XRT version
def update_server_xrt():
  print('Updating server-side XRT to {} ...'.format(CONFIG.server))
  url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
      TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
      XRT_VERSION=CONFIG.server,
  )
  print('Done updating server-side XRT: {}'.format(requests.post(url)))

update = threading.Thread(target=update_server_xrt)
update.start()

# Install Colab TPU compat PyTorch/TPU wheels and dependencies
!pip uninstall -y torch torchvision
!gsutil cp "$DIST_BUCKET/$TORCH_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCH_XLA_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCHVISION_WHEEL" .
!pip install "$TORCH_WHEEL"
!pip install "$TORCH_XLA_WHEEL"
!pip install "$TORCHVISION_WHEEL"
!sudo apt-get install libomp5
update.join()

!pip install pytorch-lightning


!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev > /dev/null
!pip install pytorch-lightning > /dev/null


!git clone https://github.com/styler00dollar/Colab-DFDNet DFDNet
%cd /content/DFDNet
!pip install -r requirements.txt
!pip install pytorch-msssim
!pip install trains
!pip install PyJWT==1.7.1
!pip install tensorboardX

In [ ]:
#@title download data
!mkdir /content/DFDNet/DictionaryCenter512
%cd /content/DFDNet/DictionaryCenter512
!gdown --id 1sEB9j3s7Wj9aqPai1NF-MR7B-c0zfTin
!gdown --id 1H4kByBiVmZuS9TbrWUR5uSNY770Goid6
!gdown --id 10ctK3d9znZ9nGN3d1Z77xW3GGshbeKBb
!gdown --id 1gcwmrIZjPFVu-cHjdQD6P4luohkPsil-
!gdown --id 1rJ8cORPxbJsIVAiNrjBag0ihaY_Mvurn
!gdown --id 1LkfJv2a3ud-mefAc1eZMJuINuNdSYgYO
!gdown --id 1LH-nxD__icSJvTiAbXAXDch03oDtbpkZ
!gdown --id 1JRTStLFsQ8dwaQjQ8qG5fNyrOvo6Tcvd
!gdown --id 1Z4AkU1pOYTYpdbfljCgNMmPilhdEd0Kl
!gdown --id 1Z4e1ltB3ACbYKzkoMBuVtzZ7a310G4xc
!gdown --id 1fqWmi6-8ZQzUtZTp9UH4hyom7n4nl8aZ
!gdown --id 1wfHtsExLvSgfH_EWtCPjTF5xsw3YyvjC
!gdown --id 1Jr3Luf6tmcdKANcSLzvt0sjXr0QUIQ2g
!gdown --id 1sPd4_IMYgqGLol0gqhHjBedKKxFAxswR
!gdown --id 1eVFjXJRnBH4mx7ZbAmZRwVXZNUbgCQec
!gdown --id 1w0GfO_KY775ZVF3KMk74ya6QL_bNU4cJ
%cd /content/DFDNet
!gdown --id 1VE5tnOKcfL6MoV839IVCCw5FhJxIgml5
!7z x data.zip
!mkdir /content/DFDNet/weights/
%cd /content/DFDNet/weights/
!gdown --id 1SfKKZJduOGhDD27Xl01yDx0-YSEkL2Aa

In [ ]:
# (Optional) EfficientNet
!pip install efficientnet_pytorch

# Train

In [ ]:
%cd /content/DFDNet/lightning
!python train.py

# Check data within dataset

Training can crash if the extracted feature has dimension 0 somewhere inside the shape. To avoid this problem, it is recommended to test all images with vgg extraction, to make sure that won't crash normal training. Test for one epoch. If a checkpoint gets generated, don't use it for real training. Broken filenames will get printed. If epoch is finished, the test is complete. Delete the printed files (and their landmarks). Data inside validation folders does not matter and will be skipped, but you probably should put at least one file there to avoid crashing. Ignore loss.

Tip: Avoid pictures with multiple faces.

Warning: The provided dataset has one bad image and will crash training because there are 2 faces. You will find one file.

In [ ]:
# File checking
%cd /content/DFDNet/lightning
!python check.py